In [3]:
import pandas as pd

train_feature = pd.read_csv("C:\\Users\\aditi\\Downloads\\summer analytic\\hack-a-thon\\training_set_features.csv")
test_features = pd.read_csv("C:\\Users\\aditi\\Downloads\\summer analytic\\hack-a-thon\\test_set_features.csv")


In [4]:
train_feature.sample(3)

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
7936,7936,2.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,dqpwygqj,"MSA, Not Principle City",0.0,1.0,fcxhlnwr,bxpfxfdn
18251,18251,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,NaN,NaN,Own,NaN,lrircsnp,"MSA, Not Principle City",0.0,0.0,NaN,NaN
10448,10448,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,"> $75,000",NaN,Own,Employed,lrircsnp,"MSA, Not Principle City",0.0,0.0,atmlpfrs,ccgxvspp


In [32]:
x_df = train_feature.drop(columns="respondent_id")
test_id = test_features["respondent_id"]
test_df = test_features.drop(columns="respondent_id")

In [7]:
x_df.head(3)

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo


In [9]:
def na_mode(df):
    for col in df.columns:
        df[col].fillna(df[col].mode()[0], inplace=True)

In [10]:
def object_num(df, col):
    mp = {}
    lst = df[col].unique()
    for idx, val in enumerate(lst):
        mp[val] = idx
    df[col] = df[col].map(mp)

In [11]:
def col_num(df):
    col_to_change = []
    for col in df.columns:
        if(df[col].dtype == "object"):
            col_to_change.append(col)

    for col in col_to_change:
        object_num(df, col)

In [12]:
na_mode(x_df)
col_num(x_df)

C:\Users\aditi\AppData\Local\Temp\ipykernel_16564\4190382441.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


In [13]:
x_df.shape


(26707, 35)

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

In [15]:

def build_model(input_shape):
    model = Sequential()
    model.add(Dense(128, input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    return model

In [16]:
input_shape = (x_df.shape[1],) 
model = build_model(input_shape)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy', AUC(name='auc')])
model.summary()

C:\Users\aditi\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         4,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,010 (58.63 KB)

 Trainable params: 15,010 (58.63 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
y_df = pd.read_csv(r"C:\Users\aditi\Downloads\summer analytic\hack-a-thon\training_set_labels.csv")
y_df.head()

,respondent_id,xyz_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


In [18]:
y_df = y_df.drop(columns="respondent_id") 
y_df.shape

(26707, 2)

In [19]:
history = model.fit(x_df, y_df, epochs=500, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/500
668/668 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.3115 - auc: 0.7128 - loss: 0.5804 - val_accuracy: 0.2911 - val_auc: 0.8247 - val_loss: 0.4763
Epoch 2/500
668/668 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3119 - auc: 0.8150 - loss: 0.4888 - val_accuracy: 0.3036 - val_auc: 0.8345 - val_loss: 0.4650
Epoch 3/500
668/668 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3045 - auc: 0.8309 - loss: 0.4714 - val_accuracy: 0.3083 - val_auc: 0.8391 - val_loss: 0.4625
Epoch 4/500
668/668 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3130 - auc: 0.8394 - loss: 0.4625 - val_accuracy: 0.2973 - val_auc: 0.8424 - val_loss: 0.4581
Epoch 5/500
668/668 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3111 - auc: 0.8390 - loss: 0.4643 - val_accuracy: 0.3051 - val_auc: 0.8442 - val_loss: 0.4526
Epoch 6/500
668/668 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3276 - auc: 0.8423 - loss: 0.4580 - val_accuracy: 0.3034 - val_auc: 0.8443 - val_loss: 0.4530
Epoch 7/500
668/668 ━━━━━━━━━━━━━━

In [21]:
na_mode(test_df)
col_num(test_df)

C:\Users\aditi\AppData\Local\Temp\ipykernel_16564\4190382441.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


In [22]:
test_df.shape


(26708, 35)

In [23]:

y_pred_probs = model.predict(test_df)
y_pred_probs

835/835 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


array([[0.05200159, 0.42657253],
       [0.05618347, 0.05740158],
       [0.38195172, 0.4153626 ],
       ...,
       [0.13509013, 0.24484077],
       [0.03082343, 0.10065903],
       [0.64243746, 0.7971107 ]], dtype=float32)

In [24]:
y_pred_probs.shape


(26708, 2)

In [25]:
y_pred = pd.DataFrame(y_pred_probs, columns=['h1n1_vaccine', 'seasonal_vaccine'])


In [26]:
y_pred.head()


,h1n1_vaccine,seasonal_vaccine
0,0.052002,0.426573
1,0.056183,0.057402
2,0.381952,0.415363
3,0.435380,0.731512
4,0.775499,0.784700


In [28]:
test_id = pd.DataFrame(test_id, columns=['respondent_id'])
test_id.head()

,respondent_id
0,26707
1,26708
2,26709
3,26710
4,26711


In [29]:
combined = pd.concat([test_id, y_pred], axis=1)
combined.shape

(26708, 3)

In [30]:
combined.to_csv('submission.csv', index=False)

In [31]:
from sklearn.metrics import roc_auc_score
results = model.evaluate(x_df, y_df, verbose=1)

y_pred_proba = model.predict(x_df)

roc_auc_xyz = roc_auc_score(y_df.iloc[:, 0], y_pred_proba[:, 0])
roc_auc_seasonal = roc_auc_score(y_df.iloc[:, 1], y_pred_proba[:, 1])

print(f"ROC AUC for XYZ Vaccine: {roc_auc_xyz}")
print(f"ROC AUC for Seasonal Vaccine: {roc_auc_seasonal}")

835/835 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3450 - auc: 0.9354 - loss: 0.3180
835/835 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
ROC AUC for XYZ Vaccine: 0.9073232998530384
ROC AUC for Seasonal Vaccine: 0.9107147751266225
